In [1]:
import pandas as pd
import numpy as np
import re

SEED = 42


from transformers import (DataCollatorWithPadding, Trainer, TrainingArguments,
                          LongformerTokenizer, LongformerForSequenceClassification,
                          LongformerConfig)

from datasets import Dataset, DatasetDict

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import torch
from torch.utils.data import DataLoader
assert torch.cuda.is_available(), 'GPU not found. You should fix this.'

In [2]:
from captum.attr import LayerConductance, LayerIntegratedGradients
from captum.attr import visualization as viz

## Data

In [3]:
def get_datadict(score_to_predict):
    
    scores = {
        'Overall',
        'Cohesion',
        'Syntax',
        'Vocabulary',
        'Phraseology',
        'Grammar',
        'Conventions'
    }
    
    columns_to_remove = scores.symmetric_difference([score_to_predict])
    
    dd = (DatasetDict
          .load_from_disk('../data/ellipse.hf')
          .remove_columns(columns_to_remove)
          .rename_column(score_to_predict, 'label')
         )
    
    return dd

In [4]:
score_to_predict = 'Grammar'

dd = get_datadict(score_to_predict)
dd

DatasetDict({
    train: Dataset({
        features: ['text_id', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4537
    })
    dev: Dataset({
        features: ['text_id', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 972
    })
    test: Dataset({
        features: ['text_id', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 973
    })
})

## Testbed

In [5]:
model_chkpt = '../bin/checkpoint-284/'
model = LongformerForSequenceClassification.from_pretrained(model_chkpt, num_labels=1).cuda()
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model.eval()
model.zero_grad()

In [6]:
from torch.utils.data import DataLoader

ds = dd.with_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

eval_dataloader = DataLoader(ds['dev'].shuffle().select(range(10)), batch_size=1)

In [7]:
def create_position_ids_from_input_ids(input_ids, padding_idx):
    """
    Replace non-padding symbols with their position numbers. Position numbers begin at padding_idx+1. Padding symbols
    are ignored. This is modified from fairseq's `utils.make_positions`.

    Args:
        x: torch.Tensor x:

    Returns: torch.Tensor
    """
    # The series of casts and type-conversions here are carefully balanced to both work with ONNX export and XLA.
    mask = input_ids.ne(padding_idx).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    return incremental_indices.long() + padding_idx

In [8]:
def predict(input_ids=None, token_type_ids=None, position_ids=None, attention_mask=None):
    
    output = model(input_ids=input_ids,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    
    return output.logits

In [9]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [10]:
for batch in eval_dataloader:
    model.zero_grad()
    
    labels = batch.pop('label')
        
    # outputs = model(**batch, output_hidden_states=True)
    
    batch['position_ids'] = create_position_ids_from_input_ids(batch['input_ids'], model.config.pad_token_id)
    
    batch = {k: v.cuda() for k, v in batch.items()}
    
    with torch.no_grad():
        score = predict(**batch)
    
    lig = LayerIntegratedGradients(predict, model.longformer.embeddings)
    
    tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][0].detach().tolist())
    
    tokens = [t.replace('Ġ', '') for t in tokens]
    
    attribution, delta = lig.attribute(inputs=batch['input_ids'],
                                       # baselines=ref_input_ids,
                                       additional_forward_args=(batch['token_type_ids'], 
                                                                batch['position_ids'],
                                                                batch['attention_mask']),
                                       return_convergence_delta=True)
    
    attribution_sum = summarize_attributions(attribution)
    
    
    # storing couple samples in an array for visualization purposes
    position_vis = viz.VisualizationDataRecord(
        attribution_sum,
        torch.max(torch.softmax(score[0], dim=0)),
        score,
        score,
        str(0),
        attribution_sum.sum(),       
        tokens,
        delta)

    print('\033[1m', 'Visualizations', '\033[0m')
    viz.visualize_text([position_vis])

 Visualizations 


 Visualizations 


 Visualizations 


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.76 GiB (GPU 0; 47.54 GiB total capacity; 43.14 GiB already allocated; 1.66 GiB free; 44.83 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF